In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('US_WeatherEvents_2016-2019.csv')

In [3]:
df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [4]:
df.describe()

,LocationLat,LocationLng,ZipCode
count,5.059833e+06,5.059833e+06,5.021148e+06
mean,3.882645e+01,-9.203908e+01,5.277158e+04
std,5.489153e+00,1.337799e+01,2.555838e+04
min,2.455710e+01,-1.245550e+02,1.022000e+03
25%,3.460690e+01,-9.800580e+01,3.172100e+04
50%,3.945200e+01,-9.011010e+01,5.450100e+04
75%,4.311110e+01,-8.215590e+01,7.364700e+04
max,4.894020e+01,-6.779280e+01,9.936200e+04


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5059833 entries, 0 to 5059832
Data columns (total 13 columns):
EventId           object
Type              object
Severity          object
StartTime(UTC)    object
EndTime(UTC)      object
TimeZone          object
AirportCode       object
LocationLat       float64
LocationLng       float64
City              object
County            object
State             object
ZipCode           float64
dtypes: float64(3), object(10)
memory usage: 501.8+ MB


In [6]:
df.isnull().sum()

EventId               0
Type                  0
Severity              0
StartTime(UTC)        0
EndTime(UTC)          0
TimeZone              0
AirportCode           0
LocationLat           0
LocationLng           0
City               9087
County                0
State                 0
ZipCode           38685
dtype: int64

In [11]:
print("The domain of Type Columns: ", df.Type.unique())
print("The domain of Severity Columns: ", df.Severity.unique())
print(len(df.AirportCode.unique()))
print(df.TimeZone.unique())
print(len(df.County.unique()))
print(len(df.State.unique()))
# print(df.EventId.unique())

The domain of Type Columns:  ['Snow' 'Fog' 'Cold' 'Storm' 'Rain' 'Precipitation' 'Hail']
The domain of Severity Columns:  ['Light' 'Severe' 'Moderate' 'Heavy' 'UNK' 'Other']
2071
['US/Mountain' 'US/Central' 'US/Eastern' 'US/Pacific']
1100
48


In [12]:
df.Type = df.Type.astype('category').cat.codes
df.Severity = df.Severity.astype('category').cat.codes
print("The domain of Type Columns: ", df.Type.unique())
print("The domain of Severity Columns: ", df.Severity.unique())

The domain of Type Columns:  [5 1 0 6 4 3 2]
The domain of Severity Columns:  [1 4 2 0 5 3]


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5059833 entries, 0 to 5059832
Data columns (total 13 columns):
EventId           object
Type              int8
Severity          int8
StartTime(UTC)    object
EndTime(UTC)      object
TimeZone          object
AirportCode       object
LocationLat       float64
LocationLng       float64
City              object
County            object
State             object
ZipCode           float64
dtypes: float64(3), int8(2), object(8)
memory usage: 434.3+ MB


In [14]:
df.drop(['ZipCode', 'City', 'LocationLat', 'LocationLng', 'EventId'], axis=1, inplace=True)

In [15]:
df.head()

,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,County,State
0,5,1,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,Saguache,CO
1,5,1,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,Saguache,CO
2,5,1,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,Saguache,CO
3,5,1,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,Saguache,CO
4,5,1,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,Saguache,CO


In [22]:
"""from datetime import datetime
import matplotlib.pyplot as plt
from datetime import datetime
import folium
from folium import plugins
from folium.plugins import MarkerCluster"""
df.describe()

,Type,Severity
count,5.059833e+06,5.059833e+06
mean,3.387343e+00,1.717773e+00
std,1.465859e+00,1.211017e+00
min,0.000000e+00,0.000000e+00
25%,3.000000e+00,1.000000e+00
50%,4.000000e+00,1.000000e+00
75%,4.000000e+00,2.000000e+00
max,6.000000e+00,5.000000e+00
